# Projeto 1 - Ciência dos Dados

Nome: Pedro de Souza Zequi

Nome: Tomás Fiorelli Barbosa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [86]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from nltk.tokenize import TweetTokenizer

In [87]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\tomas\OneDrive - Insper - Institudo de Ensino e Pesquisa\Documents\2º Semestre (2021.1) - DP\Ciência de Dados\Projetos\CDados-Projeto_1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [88]:
filename = 'Formula_1.xlsx'

In [89]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,formula e × @bts_twt https://t.co/y2gbhsrqmz,0
1,só para que saibam que já vou no 4° episódio d...,1
2,@medusabarrias eu tou tao contente que ele ten...,1
3,@sofiahoppe1 fórmula 1 é muito bom mesmo kkkkk,1
4,tava viciadão em fórmula 1 - dirigir pra viver...,1


In [90]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,fórmula 1 ??? https://t.co/y9wvxjj6gx,0
1,@enae_van 😢\nah mas fala pra eles assistirem! ...,1
2,@spluluzinha ainda nem começou a temporada de ...,1
3,melhores momentos de murray walker: comentando...,1
4,a fórmula 1 é incrível não apenas por tudo o q...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.



O projeto visa estudar comentários de internautas no Twitter sobre a Fórmula 1, o mais famoso e importante campeonato mundial de corrida de carros em que vinte dos melhores pilotos do mundo, que compõem dez times de grandes marcas automotivas, disputam corridas em diferentes circuitos do mundo com o objetivo de marcar mais pontos, pódios e vitórias e se sagrar campeão ao final da temporada.  Para classificar os tweets, a dupla se colocou em posição dos diretores da F1 que classificariam como relevantes aqueles tweets em que o conteúdo seria uma crítica, um elogio ou um comentário que poderia ser útil para o desenvolvimento do campeonato e da empresa responsável pela F1. Os classificados como irrelevantes foram aqueles tweets que não tratavam sobre Fórmula 1 ou não seriam úteis para aprimorar a imagem da empresa e do campeonato.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [91]:
tweet_tokenizer = TweetTokenizer()

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-./\|@:?;°]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def lista (serie):    
    train=""
    for frase in serie:
        train+=frase
    train=cleanup(train.lower())
    palavras=tweet_tokenizer.tokenize(train)
    return palavras

def retira_repetidas (lista):
    nova=[]
    for p in lista:
        if p not in nova:
            nova.append(p)
    return nova

# Frequências (Relevante = 1 / Irrelevante = 0)
frequencias = train.Relevância.value_counts(normalize=True)

In [92]:
# Separação dos tweets (Relevante/Irrelevante)

relevante = train.loc[train['Relevância'] == 1, ['Treinamento']]
irrelevante = train.loc[train['Relevância'] == 0, ['Treinamento']]
treinamento = train.loc[:, ['Treinamento']]

In [93]:
# Função para limpar, arrumar e tranformar em lista o tweet

def arruma_tweets(serie):
    lista_palavras = []
    frase = []
    for tweet in serie:
        frase = retira_repetidas(lista(cleanup(tweet)))
        for palavra in frase:
            lista_palavras.append(palavra)
    return lista_palavras

In [94]:
# --- Contagem das palavras ---

# Séries
serie_relevante = pd.Series(arruma_tweets(relevante.Treinamento))
serie_irrelevante = pd.Series(arruma_tweets(irrelevante.Treinamento))
serie_dicionario = pd.Series(arruma_tweets(treinamento.Treinamento))

# Frequências Absolutas
freqAbs_relevante = serie_relevante.value_counts()
freqAbs_irrelevante = serie_irrelevante.value_counts()
freqAbs_dicionario = serie_dicionario.value_counts()

# Frequêcias Relativas
freqRel_relevante = serie_relevante.value_counts(True)
freqRel_irrelevante = serie_irrelevante.value_counts(True)
freqRel_dicionario = serie_dicionario.value_counts(True)

In [103]:
# Probabilidade de Relevância de cada Palavra

prob_palavras = pd.DataFrame(data={'Relevante':freqRel_relevante, 'Irrelevante':freqRel_irrelevante})
relevancia_palavra = prob_palavras.Relevante > prob_palavras.Irrelevante

prob_palavras['Classificação'] = relevancia_palavra
prob_palavras['Classificação'] = prob_palavras['Classificação'].astype('category')
prob_palavras['Classificação'] = prob_palavras['Classificação'].cat.rename_categories([0, 1])

prob_palavras.sort_values('Relevante', ascending=False).head(10)

,Relevante,Irrelevante,Classificação
1,0.052580,0.036622,1
fórmula,0.038299,0.035874,1
de,0.030185,0.024290,1
a,0.023369,0.017190,1
e,0.019150,0.018685,1
é,0.015904,0.013079,1
formula,0.015579,0.011211,1
que,0.013632,0.014574,0
o,0.013632,0.015695,0
da,0.013307,0.010837,1


In [96]:
# Laplace Smoothing

laplace_relevante = (freqAbs_relevante + 1)/(len(freqAbs_relevante) + len(freqAbs_dicionario))
laplace_NaN_relevante = 1/(len(freqAbs_relevante) + len(freqAbs_dicionario))

laplace_irrelevante = (freqAbs_irrelevante + 1)/(len(freqAbs_irrelevante) + len(freqAbs_dicionario))
laplace_NaN_irrelevante = 1/(len(freqAbs_irrelevante) + len(freqAbs_dicionario))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [97]:
# Função de análise de relevância do Tweet

def Relevancia(tweet):
    tweet_relevante = 1
    tweet_irrelevante = 1
    for palavra in cleanup(tweet).split():
        
        # Palavra existe em relevante e irrelevante
        if palavra in laplace_relevante and palavra in laplace_irrelevante:
            tweet_relevante = tweet_relevante*laplace_relevante[palavra]
            tweet_irrelevante = tweet_irrelevante*laplace_irrelevante[palavra]
            
        # Palavra existe em relevante e não existe em irrelevante
        elif palavra in laplace_relevante and palavra not in laplace_irrelevante:
            tweet_relevante = tweet_relevante*laplace_relevante[palavra]
            tweet_irrelevante = tweet_irrelevante*laplace_NaN_irrelevante
        
        # Palavra não existe em relevante e existe em irrelevante
        elif palavra not in laplace_relevante and palavra in laplace_irrelevante:
            tweet_relevante = tweet_relevante*laplace_NaN_relevante
            tweet_irrelevante = tweet_irrelevante*laplace_irrelevante[palavra]
        
        # Palavra não existe existe em relevante e irrelevante
        else:
            tweet_relevante = tweet_relevante*laplace_NaN_relevante
            tweet_irrelevante = tweet_irrelevante*laplace_NaN_irrelevante
            
    if tweet_relevante > tweet_irrelevante:
        # Relevante
        return(1)
    else:
        # Irrelevante
        return(0)

In [98]:
# Passagem por todos os tweets e armazenando resultados obtidos

resultados = []

for tweet in test.Teste:
    resultados.append(Relevancia(tweet))

In [99]:
# Classificação final dos tweets

dicionario = {'Relevância': test['Relevância'], 'Resultado Obtido': resultados}
df_final = pd.DataFrame(data=dicionario)

classificacao = []
for linha in range(len(df_final)):
    
    # Verdadeiro positivo
    if df_final.iloc[linha][0] == df_final.iloc[linha][1] and df_final.iloc[linha][1] == 1:
        classificacao.append('Verdadeiro Positivo')
        
    # Verdadeiro negativo
    elif df_final.iloc[linha][0] == df_final.iloc[linha][1] and df_final.iloc[linha][1] == 0:
        classificacao.append('Verdadeiro Negativo')
    
    # Falso positivo
    elif df_final.iloc[linha][0] != df_final.iloc[linha][1] and df_final.iloc[linha][1] == 1:
        classificacao.append('Falso Positivo')
    
    # Falso negativo
    else:
        classificacao.append('Falso Negativo')

df_final['Classificação'] = classificacao

In [100]:
# Tabela cruzada entre relevância determinada e resultado obtido

pd.crosstab(df_final['Relevância'], df_final['Resultado Obtido'])

Resultado Obtido,0,1
Relevância,,
0,5,85
1,0,110


In [101]:
# Contagem dos resultados

df_final['Classificação'].value_counts()

Verdadeiro Positivo    110
Falso Positivo          85
Verdadeiro Negativo      5
Name: Classificação, dtype: int64

In [102]:
ved_pos = int(df_final['Classificação'].value_counts(normalize=True)['Verdadeiro Positivo']*100)
fal_pos = df_final['Classificação'].value_counts(normalize=True)['Falso Positivo']*100
ved_neg = df_final['Classificação'].value_counts(normalize=True)['Verdadeiro Negativo']*100
fal_neg = 0

print('------------------------- TOTAL ACERTO/ERRO -----------------------')
print('A porcentagem de acertos do classificador foi de {}%.'.format(ved_pos+ved_neg))
print('A porcentagem de erros do classificador foi de {}%.'.format(fal_pos))
print()
print('------------------------- TOTAL CLASSIFICAÇÃO -----------------------')
print('A porcentagem de verdadeiros positivos do classificador foi de {}%.'.format(ved_pos))
print('A porcentagem de falsos positivos do classificador foi de {}%.'.format(fal_pos))
print('A porcentagem de verdadeiros negativos do classificador foi de {}%.'.format(ved_neg))
print('A porcentagem de falsos negativos do classificador foi de {}%.'.format(fal_neg))

------------------------- TOTAL ACERTO/ERRO -----------------------
A porcentagem de acertos do classificador foi de 57.5%.
A porcentagem de erros do classificador foi de 42.5%.

------------------------- TOTAL CLASSIFICAÇÃO -----------------------
A porcentagem de verdadeiros positivos do classificador foi de 55%.
A porcentagem de falsos positivos do classificador foi de 42.5%.
A porcentagem de verdadeiros negativos do classificador foi de 2.5%.
A porcentagem de falsos negativos do classificador foi de 0%.


___
### Concluindo

Com a finalização do projeto 1 de Ciência de Dados, pudemos aprender diversas habilidades e conceitos quanto à análise exploratória de frases extraidas do Twitter de um tema de nossa escolha. Como resultados, obtivemos os seguintes números:

Acertos:
- Verdadeiros Positivos: **55%**
- Verdadeiros Negativos: **2.5%**
   - Total: **57.5%**

Erros:
- Falsos Positivos: **42.5%**
- Falsos Negativos: **0%**
   - Total: **42.5%**

Interpretando os resultados acima, podemos perceber que, embora se obteve uma porcentagem de acertos maior que a de erros, a diferença não é considerável a ponto de tornar o classificador em um mecanismo capaz de fazer tal atividade de maneira eficaz. O motivo de obtermos tais resultados está relacionado ao nosso critério de classificação de relevância dos tweets de treinamento, no qual era baseado em críticas construtivas, elogios e comentários que seriam úteis para entender a imagem do camponato de corrida como relevante.

O classificador considerou como relevante, praticamente, todos os tweets em que o conjunto de palavras "fórmula" e "1" aparecem (as palavras que tiveram mais ocorrências). Ou seja, muitos comentários considerados irrelevantes pela dupla foram classificados como relevantes simplesmente pelo fato de possuirem o conjuto de palavras citadas anteriormente.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

x

x

x

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**